In [ ]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
working_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if working_dir not in sys.path:
    sys.path.insert(0, working_dir)

# Personnal import 
from utils.utilities_DL import match_period_coverage_with_netmob,get_small_ds
from constants.config import get_args,update_modif
from constants.paths import FOLDER_PATH,FILE_NAME,SAVE_DIRECTORY
from K_fold_validation.K_fold_validation import KFoldSplitter
from trainer import Trainer
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from plotting.plotting_bokeh import plot_bokeh


# Load config
model_name = 'STGCN' #'CNN'
args = get_args(model_name)

# Modification : 
args.K_fold = 5

args.ray = False
args.W = 0  # IMPORTANT AVEC NETMOB

args.epochs = 100
args.loss_function_type = 'MSE' # 'quantile'

# optimization:
args.mixed_precision = True

args = update_modif(args)

# Coverage Period : 
small_ds = False
coverage = match_period_coverage_with_netmob(FILE_NAME,dataset_names=['subway_in','netmob'])
(coverage,args) = get_small_ds(small_ds,coverage,args)

# Choose DataSet and VisionModel if needed: 
dataset_names = ['netmob','subway_in'] # ['calendar','netmob'] #['subway_in','netmob','calendar']
vision_model_name = 'FeatureExtractor_ResNetInspired'  # 'ImageAvgPooling'  #'FeatureExtractor_ResNetInspired' #'MinimalFeatureExtractor',

# Train and Evaluate Model: 
mod_plot = 1 # bokeh plotting every epoch 

# Load K-fold subway-ds 
folds = [0] # Here we use the first fold for HP-tuning. 

# In case we need to compute the Sliding K-fold validation:
# folds = np.arange(1,args.K_fold)

K_fold_splitter = KFoldSplitter(dataset_names,args,coverage,vision_model_name,folds)
K_subway_ds,dic_class2rpz = K_fold_splitter.split_k_fold()
subway_ds = K_subway_ds[0]

In [ ]:
# Load Model, trainer, and train it:
model = load_model(args,dic_class2rpz)
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Plotting: 
pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

In [ ]:
# Load Model, trainer, and train it:
model = load_model(args,dic_class2rpz)
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model,args)
trainer = Trainer(subway_ds,model,args,optimizer,loss_function,scheduler = scheduler,dic_class2rpz = dic_class2rpz,show_figure = True)# Ajoute dans trainer, if calibration_prop is not None .... et on modifie le dataloader en ajoutant un clabration set
trainer.train_and_valid(mod = 1000,mod_plot = None)  # Récupère les conformity scores sur I1, avec les estimations faites precedemment 

# Plotting: 
pi,pi_cqr = plot_bokeh(trainer,subway_ds.normalizer,subway_ds.tensor_limits_keeper.df_verif_test,args)

In [10]:
import torch

netmob_init = torch.Tensor([7392*4*263*287])
netmob_tensor = torch.Tensor([2934*4*263*287*8])

print('Init tensor: ',netmob_init)
print('Init feature vect: ',netmob_tensor)

print('\nblock1:')
sconv = torch.prod(torch.Tensor([64, 32, 131, 143, 8]))
tconv = torch.prod(torch.Tensor([64, 128, 131, 143, 8]))
print('s_conv: ',sconv, 'tconv: ',tconv)

print('\nblock2:')
sconv = torch.prod(torch.Tensor([64, 658, 65, 71, 8]))
tconv = torch.prod(torch.Tensor([64, 512, 65, 71, 8]))
print('s_conv: ',sconv, 'tconv: ',tconv)


Init tensor:  tensor([2.2318e+09])
Init feature vect:  tensor([7.0868e+09])

block1:
s_conv:  tensor(3.0692e+08) tconv:  tensor(1.2277e+09)

block2:
s_conv:  tensor(1.5548e+09) tconv:  tensor(1.2098e+09)
